- 环境配置及路径
    - `export HF_HOME='/media/whaow/.cache/huggingface'`
    - model path: `HF_HOME/hub/models--xx-xx`
        - models--meta-llama--Llama-2-7b-hf: `meta-llama/Llama-2-7b-hf`

## dtypes

In [2]:
from transformers.models.gpt2.modeling_gpt2 import GPT2Block
from transformers import AutoConfig

In [6]:
config = AutoConfig.from_pretrained('gpt2-medium')
gpt2_block = GPT2Block(config, layer_idx=0)

In [9]:
# default torch.float32
next(gpt2_block.parameters()).dtype

torch.float32

In [11]:
config = AutoConfig.from_pretrained('gpt2-medium')
gpt2_block = GPT2Block(config, layer_idx=0)
# half: 半精度
gpt2_block = GPT2Block(config, layer_idx=0).half()
next(gpt2_block.parameters()).dtype

torch.float16

## 加载本地文件

In [1]:
from transformers import AutoConfig

In [3]:
config = AutoConfig.from_pretrained('/home/whaow/.cache/huggingface/hub/models--gpt2-medium/snapshots/425b0cc90498ac177aa51ba07be26fc2fea6af9d/config.json')
config

GPT2Config {
  "_name_or_path": "/home/whaow/.cache/huggingface/hub/models--gpt2-medium/snapshots/425b0cc90498ac177aa51ba07be26fc2fea6af9d/config.json",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1024,
  "n_head": 16,
  "n_inner": null,
  "n_layer": 24,
  "n_positions": 1024,
  "n_special": 0,
  "predict_special_tokens": true,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version":

## single, half, double

- `--fp16` 与 `--bf16`
    - single（单精度）: `fp16 == False and bf16 == False`
    - half（半精度）: `fp16 == False and bf16 == True`